#ECSE 415 Assignment 3 Part 2
Hanwen Wang 260778557

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# make path = './' connect to the path of the folder stroing images
path = '/content/drive/My Drive/ECSE_415_F_2020/Assignment_3/'

Mounted at /content/drive


# 2. Image Classfication with Convolution Neural Network (CNN)

## 2.1

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

#normalize the value between -1 to 1
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5], [0.5])])

trainset = torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,shuffle=True,num_workers=2)

testset = torchvision.datasets.MNIST(root='./data',train=False,download=True,transform=transform)
testloader = torch.utils.data.DataLoader(testset,batch_size=32,shuffle=False,num_workers=2)

## 2.2

In [4]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #convolution layer
        #nn.Con2d(input channel,output channel(kernel),size), 
        #size is [,], can be simplified to a number if two numbers are equal
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 64, 3)
        #maxpool layer
        #nn.MaxPool2d(kernel size)
        self.pool = nn.MaxPool2d(2, 2)
        #linear layer
        #nn.Linear(input neurons/size,output neurons/size)
        self.fc = nn.Linear(4096, 10)
    def forward(self, x):
        #A convolution layer with 32 kernels of size 3×3 and A ReLU activation
        x = F.relu(self.conv1(x))
        #A convolution layer with 64 kernels of size 3×3, A ReLU activation and 
        #A maxpool layer with kernels of size 2×2
        x = self.pool(F.relu(self.conv2(x)))
        #A convolution layer with 64 kernels of size 3×3 and A ReLU activation
        x = F.relu(self.conv3(x))
        #A convolution layer with 64 kernels of size 3×3 and A ReLU activation
        x = F.relu(self.conv3(x))
        #vectorize into a long feature vector, a flattening layer
        x = x.view(-1,4096)
        #A Linear layer with output size of 10
        x = self.fc(x)
        return x

net = Net()

## 2.3

In [5]:
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

cuda:0


## 2.4

In [6]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 300 == 299:    # print every 300 mini-batches, 1800+ in total in one epoch
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

print('Finished Training')

[1,   300] loss: 2.297
[1,   600] loss: 2.282
[1,   900] loss: 2.254
[1,  1200] loss: 2.175
[1,  1500] loss: 1.802
[1,  1800] loss: 0.872
[2,   300] loss: 0.530
[2,   600] loss: 0.474
[2,   900] loss: 0.418
[2,  1200] loss: 0.407
[2,  1500] loss: 0.395
[2,  1800] loss: 0.392
[3,   300] loss: 0.365
[3,   600] loss: 0.371
[3,   900] loss: 0.356
[3,  1200] loss: 0.369
[3,  1500] loss: 0.369
[3,  1800] loss: 0.344
[4,   300] loss: 0.354
[4,   600] loss: 0.326
[4,   900] loss: 0.309
[4,  1200] loss: 0.318
[4,  1500] loss: 0.319
[4,  1800] loss: 0.315
[5,   300] loss: 0.301
[5,   600] loss: 0.292
[5,   900] loss: 0.284
[5,  1200] loss: 0.296
[5,  1500] loss: 0.290
[5,  1800] loss: 0.281
[6,   300] loss: 0.267
[6,   600] loss: 0.252
[6,   900] loss: 0.263
[6,  1200] loss: 0.249
[6,  1500] loss: 0.244
[6,  1800] loss: 0.236
[7,   300] loss: 0.215
[7,   600] loss: 0.229
[7,   900] loss: 0.220
[7,  1200] loss: 0.215
[7,  1500] loss: 0.198
[7,  1800] loss: 0.187
[8,   300] loss: 0.182
[8,   600] 

## 2.5

In [7]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on test images: %d %%' % (100 * correct / total))

Accuracy of the network on test images: 97 %
